# Men from Mars, Women from Venus: both can code .NET in Jupyter

@rondagdag

### .NET Interactive https://github.com/dotnet/interactive

### To install in raspberry pi
```
curl -L https://raw.githubusercontent.com/colombod/pi-top/master/setup-device.sh | bash -e
source .jupyter_venv/bin/activate
jupyter lab --no-browser

```

Rasperry Pi Sense Hat installation instructions
https://www.raspberrypi.org/documentation/hardware/sense-hat/

In [1]:
#!lsmagic

The below script needs to be able to find the current output cell; this is an easy method to get it.

.NET kernel #!csharp , #!c# , #!C# Run the code that follows using the csharp kernel. #!fsharp , #!f# , #!F# Run the code that follows using the fsharp kernel. #!pwsh , #!powershell Run the code that follows using the pwsh kernel. #!javascript , #!js Run the code that follows using the javascript kernel. #!html Run the code that follows using the html kernel. #!value Stores a value #!lsmagic List the available magic commands / directives #!markdown Convert the code that follows from Markdown into HTML #!time Time the execution of the following code in the submission. #!log Enables session logging. #!about Show version and build information #!connect Connects additional subkernels named-pipe Connects to a kernel using named pipes signalr Connects to a kernel using SignalR

csharp kernel #i <null> #r <null> #!whos , #!who Display the names of the current top-level variables and their values. #!share Share a .NET variable between subkernels #!lsmagic List the available magic commands / directives

fsharp kernel #i <null> #r <null> #!whos , #!who Display the names of the current top-level variables and their values. #!share Share a .NET variable between subkernels #!lsmagic List the available magic commands / directives

pwsh kernel #!share Share a .NET variable between subkernels #!lsmagic List the available magic commands / directives

javascript kernel #!lsmagic List the available magic commands / directives

html kernel #!lsmagic List the available magic commands / directives

value kernel #!lsmagic List the available magic commands / directives

In [119]:
#!fsharp

"Hello from F#" |> Console.WriteLine
let x = 10
let y = 15

Hello from F#


In [123]:
#!share --from fsharp x
x

10

In [6]:
#r "nuget:System.Device.Gpio,1.1.0-prerelease.20276.1"
#r "nuget:IoT.Device.Bindings,1.1.0-prerelease.20276.1"
#r "nuget:UnitsNet,4.68.0"

### Display the html

In [7]:
var pocketView = span(a[href:@"https://github.com/rondagdag/dotnetcode-jupyter-talk", target: "blank"](b("Presentation")));
display(pocketView)

Presentation

In [8]:
var someHtml = HTML("<b style=\"color:blue\">Sensor data!</b>");
var tempOutput = display(someHtml);

101.12750091552734

In [9]:
using Iot.Device.SenseHat;
using System;
var temperatures = new List<(int idx, double val)>();

async Task readTemperature(int count = 10){   
    using (var sh = new SenseHat())
    {
        var initcount = temperatures.Count;
        for (var i = initcount; i < initcount + count; i++){        
            temperatures.Add((i, sh.Temperature2.DegreesFahrenheit));
            tempOutput.Update(HTML($"<b style=\"color:green\">{sh.Temperature2.DegreesFahrenheit}</b>"));
            await Task.Delay(500);
        }
    }
}

### Read Temperatures

In [10]:
await readTemperature(15);

In [11]:
using XPlot.Plotly;

In [12]:
var values = temperatures.Select(i => i.val);
values

index,value
0,101.07499999999999
1,101.08999633789061
2,101.09375228881835
3,101.10125045776365
4,101.10499954223631
5,101.09375228881835
6,101.09750137329101
7,101.10874862670897
8,101.10499954223631
9,101.11249771118163


In [13]:
var openSeries = new Graph.Scatter
{
    name = "Open",
    x = temperatures.Select(i => i.idx),
    y = values
};


var chart = Chart.Plot(new[] {openSeries});
chart.WithTitle("Open");

### Display the chart

In [14]:
display(chart)

### Instead of adding to list, let's setup Asycnronous Streams in C#

In [15]:
async IAsyncEnumerable<double> ReadTemperatureAsync(int count = 10){   
    using (var sh = new SenseHat())
    {
        for (var i = 0; i < count; i++){
            await Task.Delay(1000);
            yield return (sh.Temperature2.DegreesFahrenheit);
        }
    };
}

In [16]:
#r "nuget:Microsoft.Data.Analysis"
using Microsoft.Data.Analysis;

Installed package Microsoft.Data.Analysis version 0.4.0

In [18]:
var output = display("Counting...");
PrimitiveDataFrameColumn<DateTime> dateTimes = new PrimitiveDataFrameColumn<DateTime>("DateTimes"); // Default length is 0.
PrimitiveDataFrameColumn<double> fahrenheit = new PrimitiveDataFrameColumn<double>("Fahrenheit"); // Makes a column of length 3. Filled with nulls initially

await foreach (var result in ReadTemperatureAsync(15))
{
   output.Update(result);
   dateTimes.Append(DateTime.Now);
   fahrenheit.Append(result);
}

101.60750045776366

### create a new Dataframe with 2 columns

In [19]:
DataFrame df =  new DataFrame(dateTimes, fahrenheit);

In [20]:
df

Columns,Rows
"[ [ 2020-08-12 07:29:34Z, 2020-08-12 07:29:35Z, 2020-08-12 07:29:36Z, 2020-08-12 07:29:37Z, 2020-08-12 07:29:38Z, 2020-08-12 07:29:39Z, 2020-08-12 07:29:40Z, 2020-08-12 07:29:41Z, 2020-08-12 07:29:42Z, 2020-08-12 07:29:43Z, 2020-08-12 07:29:44Z, 2020-08-12 07:29:45Z, 2020-08-12 07:29:46Z, 2020-08-12 07:29:47Z, 2020-08-12 07:29:48Z ], [ 101.60750045776366, 101.61124954223632, 101.63000183105467, 101.63000183105467, 101.63375091552733, 101.63749999999999, 101.63000183105467, 101.61874771118163, 101.61499862670897, 101.62249679565429, 101.62249679565429, 101.61124954223632, 101.61124954223632, 101.61874771118163, 101.60750045776366 ] ]","[ [ 2020-08-12 07:29:34Z, 101.60750045776366 ], [ 2020-08-12 07:29:35Z, 101.61124954223632 ], [ 2020-08-12 07:29:36Z, 101.63000183105467 ], [ 2020-08-12 07:29:37Z, 101.63000183105467 ], [ 2020-08-12 07:29:38Z, 101.63375091552733 ], [ 2020-08-12 07:29:39Z, 101.63749999999999 ], [ 2020-08-12 07:29:40Z, 101.63000183105467 ], [ 2020-08-12 07:29:41Z, 101.61874771118163 ], [ 2020-08-12 07:29:42Z, 101.61499862670897 ], [ 2020-08-12 07:29:43Z, 101.62249679565429 ], [ 2020-08-12 07:29:44Z, 101.62249679565429 ], [ 2020-08-12 07:29:45Z, 101.61124954223632 ], [ 2020-08-12 07:29:46Z, 101.61124954223632 ], [ 2020-08-12 07:29:47Z, 101.61874771118163 ], [ 2020-08-12 07:29:48Z, 101.60750045776366 ] ]"


In [21]:
using Microsoft.AspNetCore.Html;
Formatter<DataFrame>.Register((df, writer) =>
{
    var headers = new List<IHtmlContent>();
    headers.Add(th(i("index")));
    headers.AddRange(df.Columns.Select(c => (IHtmlContent) th(c.Name)));
    var rows = new List<List<IHtmlContent>>();
    var take = 5;
    for (var i = 0; i < Math.Min(take, df.Rows.Count); i++)
    {
        var cells = new List<IHtmlContent>();
        cells.Add(td(i));
        foreach (var obj in df.Rows[i])
        {
            cells.Add(td(obj));
        }
        rows.Add(cells);
    }
    
    var t = table(
        thead(
            headers),
        tbody(
            rows.Select(
                r => tr(r))));
    
    writer.Write(t);
}, "text/html");

In [22]:
df

0,2020-08-12 07:29:34Z,101.60750045776366
1,2020-08-12 07:29:35Z,101.61124954223632
2,2020-08-12 07:29:36Z,101.63000183105467
3,2020-08-12 07:29:37Z,101.63000183105467
4,2020-08-12 07:29:38Z,101.63375091552733


In [23]:
var a = df.Rows.Take(2);
a

index,value
0,"[ 2020-08-12 07:29:34Z, 101.60750045776366 ]"
1,"[ 2020-08-12 07:29:35Z, 101.61124954223632 ]"


### Count the number of rows

In [24]:
var total = df.Rows.Count();
total

15

In [25]:
PrimitiveDataFrameColumn<double> celcius = new PrimitiveDataFrameColumn<double>("Celcius", total);
var newDf = df.Clone();
newDf.Columns.Insert(2, celcius);
newDf

0,2020-08-12 07:29:34Z,101.60750045776366,<null>
1,2020-08-12 07:29:35Z,101.61124954223632,<null>
2,2020-08-12 07:29:36Z,101.63000183105467,<null>
3,2020-08-12 07:29:37Z,101.63000183105467,<null>
4,2020-08-12 07:29:38Z,101.63375091552733,<null>


In [26]:
var info = newDf.Info();
info

0,DataType,System.DateTime,System.Double,System.Double
1,Length (excluding null values),15,15,0


In [27]:
celcius.FillNulls(0, inPlace: true);
newDf

0,2020-08-12 07:29:34Z,101.60750045776366,0
1,2020-08-12 07:29:35Z,101.61124954223632,0
2,2020-08-12 07:29:36Z,101.63000183105467,0
3,2020-08-12 07:29:37Z,101.63000183105467,0
4,2020-08-12 07:29:38Z,101.63375091552733,0


In [28]:
DataFrameRow row0 = newDf.Rows[0];
row0

index,type,value
0,System.DateTime,2020-08-12 07:29:34Z
1,System.Double,101.60750045776366
2,System.Double,0


### Convert Fahrenheit and fill the Celcius

In [29]:
newDf.Columns["Celcius"] = (fahrenheit - 32) * 5/9;
newDf

0,2020-08-12 07:29:34Z,101.60750045776366,38.67083358764648
1,2020-08-12 07:29:35Z,101.61124954223632,38.672916412353516
2,2020-08-12 07:29:36Z,101.63000183105467,38.68333435058592
3,2020-08-12 07:29:37Z,101.63000183105467,38.68333435058592
4,2020-08-12 07:29:38Z,101.63375091552733,38.68541717529296


In [30]:
var celciusMean = newDf.Columns["Celcius"].Mean();
celciusMean

38.678055318196606

In [31]:
Chart.Plot(
    new Graph.Scatter()
    {
        x = newDf.Columns["DateTimes"],
        y = newDf.Columns["Celcius"]
    }
)

In [32]:
var tempData = new List<double>();

async Task readTempData(int count = 10){   
    using (var sh = new SenseHat())
    {
        for (var i = 0; i < count; i++){        
            if(tempData.Count > 10){
                tempData.RemoveAt(0);
            }
            tempData.Add(sh.Temperature2.DegreesFahrenheit);
            Console.WriteLine(sh.Temperature2.DegreesFahrenheit);
            await Task.Delay(500);
        }
    }
}

In [108]:
#!js 

if (typeof (notebookScope.interval) !== 'undefined') {
    clearInterval(notebookScope.interval);
}

notebookScope.plot = (sgvSelector, variableName) => {   
    let dtree_require = require.config({
        paths: {
            d3: "https://d3js.org/d3.v5.min"
        }
    });
    dtree_require(["d3"], function (d3) {       
        let svg = d3.select(sgvSelector);
        svg.selectAll("defs").remove();
        svg.selectAll("g").remove();
        
        var margin = {top: 0, right: 30, bottom: 30, left: 40},
            width = 260 - margin.left - margin.right,
            height = 200 - margin.top - margin.bottom;
        
        let container = d3
            .select(sgvSelector);
        
         var width = 260 
            scaleFactor = 20, 
            barHeight = 30;
         
         var graph = d3.select("body")
            .append("svg")
                .attr("width", width + margin.left + margin.right)
                .attr("height", height + margin.top + margin.bottom)
            .append("g")
                .attr("transform",
                      "translate(" + margin.left + "," + margin.top + ")");
        
        // Add X axis
          var x = d3.scaleLinear()
            .domain([0, 10])
            .range([ width, 0 ]);
        
          svg.append("g")
            .attr("transform", "translate(" + margin.left + "," + height + ")")
            .call(d3.axisBottom(x));

      // Add Y axis
          var y = d3.scaleLinear()
            .domain([101, 102])
            .range([ height, 0]);
        
          svg.append("g")
            .attr("transform", "translate(" + margin.left + ", 0)")
            .call(d3.axisLeft(y));

        updateD3Rendering = () => interactive.csharp.getVariable(variableName)
            .then(data => {
                console.log(data);
                container
                    .selectAll(".points")
                    .data(data, (d, i) => i)
                    .join(
                        enter => enter.append("circle")
                            .attr("cx", (d, i) => { console.log("i:" + i); return margin.left + x(i); } )
                            .attr("cy", function (d) { console.log("d:" + d); return y(d); } )
                            .attr("r", 2.5)
                            .style("fill", "#69b3a2")
                            .transition()
                            .duration(2000)                            
                            .attr("r", 0 ).remove(),
                        update => update
                            .transition()
                             .attr("class", "line")
                             .attr("d", valueline)
                            );
            });
        notebookScope.interval = setInterval(() => updateD3Rendering(), 1000);
    });
}

### Plot and call javascript 

In [109]:
#!html
<svg id="dataPlot1" width=500 height=200>
</svg>

#!js
notebookScope.plot("svg#dataPlot1", "tempData")

In [110]:
await readTempData(50);

101.92999725341795
101.91874999999999
101.91125183105467
101.92249908447265
101.9262481689453
101.94125137329101
101.94874954223631
101.96375274658202
101.97125091552734
101.96750183105468
101.96375274658202
101.93375320434569
101.91125183105467
101.91500091552733
101.93375320434569
101.94874954223631
101.95249862670897
101.97125091552734
101.9824981689453
101.98624725341796
101.975
101.96375274658202
101.95249862670897
101.94500045776365
101.93750228881835
101.92999725341795
101.92249908447265
101.90750274658203
101.89249954223632
101.88125228881835
101.87374725341796
101.87374725341796
101.89249954223632
101.90750274658203
101.9262481689453
101.92999725341795
101.93750228881835
101.93375320434569
101.95249862670897
101.95249862670897
101.95624771118163
101.95999679565429
101.95249862670897
101.95999679565429
101.96375274658202
101.96375274658202
101.96750183105468
101.97125091552734
101.95624771118163
101.96375274658202


In [96]:
tempData.Count

11

In [46]:
using System;
using System.Drawing;
async Task readTempCondition(int count = 10, double maxTemp = 100){   
    using (var ledMatrix = new SenseHatLedMatrixI2c())
    using (var sh = new SenseHat())
    {
        ledMatrix.Fill(Color.Black);
        for (var i = 0; i < count; i++){
            var temp = sh.Temperature2.DegreesFahrenheit;
            Console.WriteLine(temp);
            if (temp > maxTemp) {
                ledMatrix.Fill(Color.Red);
            } 
            await Task.Delay(500);
        }
    }
}

### read temperature 50 times and add max temp condition

In [49]:
await readTempCondition(15, 101.7);

101.67124862670897
101.68625183105468
101.69000091552734
101.72374954223632
101.74250183105468
101.73875274658202
101.74625091552733
101.73875274658202
101.74625091552733
101.73499679565428
101.716251373291
101.716251373291
101.70499725341796
101.7012481689453
101.69749908447264


### clear the ledMatrix to "black"

In [50]:
using (var ledMatrix = new SenseHatLedMatrixI2c())
{
        ledMatrix.Fill(Color.Black);
}


## @rondagdag

## https://www.linkedin.com/in/rondagdag